In [1]:
from pathlib import Path
import pandas as pd
import xlwings as xw
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from rdp import rdp

In [2]:
#Access input folder
input_dir1 = Path ("G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/TNT_near_field_ascii_files2")
print ("1",input_dir1)

#Access folders inside input folder
input_dir2 =  [folder for folder in input_dir1.iterdir() if folder.is_dir()] # L13_1
print ("2",input_dir2)

# Create output directory
output_dir1 = Path("G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/Near_field_time_history_dataset2")
output_dir1.mkdir(exist_ok = True)

# Define current work directory
current_dir = Path.cwd()

# Access the template to be used to generate files
excel_template = current_dir/ "Template.xlsx"

1 G:\Chamodi\LSDYNA_1D\Incident_pressure_time_history\TNT_near_field_ascii_files2
2 [WindowsPath('G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/TNT_near_field_ascii_files2/L13_13'), WindowsPath('G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/TNT_near_field_ascii_files2/L13_17'), WindowsPath('G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/TNT_near_field_ascii_files2/L13_5'), WindowsPath('G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/TNT_near_field_ascii_files2/L13_9')]


In [3]:
def reduce_points_CT (time, pressure):
    
    zero_crossing_index = next((i for i, val in enumerate(pressure[:], start=0) if val <= 0), None)
    # CHANGE
    CT = time[zero_crossing_index]
    CP = pressure[zero_crossing_index]
    
    # Combine time and pressure into a single array of 2D points
    points = np.column_stack((time, pressure))
    
    segment1 = points[:zero_crossing_index]  # Example segmentation
    segment2 = points[zero_crossing_index:]
    
    reduced_segment1 = rdp(segment1, epsilon=0.001)
    reduced_segment2 = rdp(segment2, epsilon=0.01)
    
    reduced_points = np.vstack((reduced_segment1, reduced_segment2))
    
    # Separate the reduced points into time and pressure
    reduced_time = reduced_points[:, 0]
    reduced_pressure = reduced_points[:, 1]
    
    return reduced_time, reduced_pressure, CT, CP

In [4]:
def get_phase_array(time, pressure, CT, CP):
    a=0
    phase = []
    for t, p in zip(time, pressure):
        if round (t, 10) < round (CT, 10):
            phase.append('positive')#{t} {change_time}')
        elif round (CT, 10) <= round (t, 10):
            a+=1
            if a == 1 and abs(p - CP) <0.000001:
                phase.append('negative')
            elif a == 1 and abs(p - CP) >0.000001:
                phase.append('error')
                print (f'error- p={p} , change p ={CP}, t ={t}, change t = {CT}')
            else:
                phase.append('negative')
        else:
            phase.append('something wrong')
            print ('something wrong')
                
    return phase

In [5]:
df_all = pd.DataFrame(columns=["Mass", "Standoff distance", "Arrival time", "AP", "Change time", "CP", "End time", "EP"])

for folder1 in input_dir2: # folder1-L13_1
    
    # Create output folders
    output_dir2 = output_dir1/ folder1.name
    output_dir2.mkdir(exist_ok = True)
    print ("OUTPUT 2", output_dir2)
    
    # EM
    parts = folder1.name.split('_')
    EM = round(int(parts[1])/2, 1)
    
    # Make a list of data file names
    txt_files = [file for file in folder1.rglob("*") if file.name.isdigit() and int(file.name) % 2 != 0]
    
    # Access each file 
    for file1 in txt_files: # file1-1
        
        # Assign value to A based on file.stem
        file_name = file1.stem
        SD_mapping = {"1": 1, "3": 1.25, "5":1.5, "7":1.75, "9":2,
                      "11":2.25, "13":2.5, "15":2.75, "17":3}
        SD = SD_mapping.get(file_name, "Unknown")
        
        # Read data from Excel file
        df1 = pd.read_csv(file1,
                          delimiter="\s+",
                          skiprows = 8,
                          skipfooter = 1,
                          engine = 'python',
                          header = None,
                          names = ["Time", "Pressure"])
        
        # Extract time and pressure columns
        org_time = df1["Time"].dropna().values*1000
        org_pressure = (df1["Pressure"].dropna().values/1000)-101.2
        
        # Find indices where org_time is greater than 29 seconds (29000 milliseconds)
        indices_to_remove = np.where(org_time > 29)[0]
        
        # Remove the corresponding values in org_time and org_pressure
        time = np.delete(org_time, indices_to_remove)
        pressure = np.delete(org_pressure, indices_to_remove)
            
        peak_index = np.argmax(pressure)
        
        # ARRIVAL
        AT = time[peak_index]
        AP = pressure[peak_index]
        
        pressure = pressure[peak_index:]
        time = time[peak_index:]
        
        for i in range(len(pressure) - 1, -1, -1):
            if pressure[i] < -1:
                end_index = i
                break
        # END
        ET = time[end_index]
        EP = pressure[end_index]
        
        pressure = pressure[:end_index + 1]
        time = time[:end_index + 1]        

        reduced_time, reduced_pressure, CT, CP =reduce_points_CT(time, pressure)
        
        if all(isinstance(val, np.floating) for val in [CT, CP]):
            phase = get_phase_array(reduced_time, reduced_pressure, CT, CP)
            
            # Create a dictionary with the arrays
            data = {"Mass": [EM] * len(reduced_time),
                     "Standoff distance": [SD] * len(reduced_time),
                     'Time': reduced_time,
                     "Phase": phase,
                     'Pressure': reduced_pressure}
            
            # Create DataFrames from dictionaries
            df2 = pd.DataFrame(data)
            
            # Initiate xlwings library
            with xw.App (visible = False) as app:
                
                wb = app.books.open(excel_template)
                # Write dataframe in excel template
                wb.sheets[0].range("A1").options(index=False).value = df2
                
                # Create files in output directory
                wb.save(output_dir2/f"{file_name}.xlsx")
                wb.close()
                print ("file_name",file_name)
            
            # Create the plot
            plt.figure(figsize=(4, 4))
            plt.plot(org_time, org_pressure, color = 'g', linestyle = '-', linewidth=1, label='Original')
            plt.plot(time, pressure, color='b', linestyle=':', linewidth=2, label='Trimmed')
            plt.plot(reduced_time, reduced_pressure, color='r', linestyle='-', marker='o', linewidth=1, label='Reduced')
            plt.legend()
            # Save the plot as an image
            plt.savefig(output_dir2/f'{file_name}.png', dpi=150, bbox_inches='tight', facecolor='white')
            
                
            # Close the plot to free up memory
            plt.close() 
            
        else:
            CT = CT if isinstance(CT, (np.floating, float)) else None
            CP = CP if isinstance(CP, (np.floating, float)) else None
            

        # Creating a new DataFrame to append
        row_all = pd.DataFrame({"Mass": [EM], "Standoff distance": [SD],
                                "Arrival time": [AT],"AP": [AP],
                                "Change time": [CT], "CP": [CP],
                                "End time": [ET], "EP": [EP],
                                "Original":[len(pressure)], "Reduced": [len(reduced_pressure)]})
        df_all = pd.concat([df_all, row_all], ignore_index=True)
                        

OUTPUT 2 G:\Chamodi\LSDYNA_1D\Incident_pressure_time_history\Near_field_time_history_dataset2\L13_13
file_name 1
file_name 11
file_name 13
file_name 15
file_name 17
file_name 3
file_name 5
file_name 7
file_name 9
OUTPUT 2 G:\Chamodi\LSDYNA_1D\Incident_pressure_time_history\Near_field_time_history_dataset2\L13_17
file_name 1
file_name 11
file_name 13
file_name 15
file_name 17
file_name 3
file_name 5
file_name 7
file_name 9
OUTPUT 2 G:\Chamodi\LSDYNA_1D\Incident_pressure_time_history\Near_field_time_history_dataset2\L13_5
file_name 1
file_name 11
file_name 13
file_name 15
file_name 17
file_name 3
file_name 5
file_name 7
file_name 9
OUTPUT 2 G:\Chamodi\LSDYNA_1D\Incident_pressure_time_history\Near_field_time_history_dataset2\L13_9
file_name 1
file_name 11
file_name 13
file_name 15
file_name 17
file_name 3
file_name 5
file_name 7
file_name 9


In [6]:
# Write dataframes in excel template
all_path = 'IP_near_field_all_reduced2.xlsx'

df_all.to_excel(all_path, index=False)